In [1]:
!pip install --disable-pip-version-check python-dotenv web3 eth-account hexbytes vyper==0.3.10

In [2]:
import os, re, time, textwrap
from pathlib import Path
from dotenv import load_dotenv, set_key
from eth_account import Account
from web3 import Web3
from vyper import compile_codes

In [3]:
ENV_PATH = r"C:\Users\aniru\OneDrive\Desktop\ML tutorial\DEFI Depeg sentinel\python\.env"

In [4]:
AUTO_WAIT_FOR_FUNDS = True
POLL_EVERY_SECS = 8
GAS_LIMIT_DEPLOY = 500_000
GAS_LIMIT_WRITE  = 200_000

In [5]:
RPC_CANDIDATES = [
    "https://ethereum-sepolia-rpc.publicnode.com",
    "https://1rpc.io/sepolia",
    "https://rpc.sepolia.org",
    "https://rpc2.sepolia.org",
]
CHAIN_ID = 11155111

In [17]:
Path(ENV_PATH).parent.mkdir(parents=True, exist_ok=True)
if not Path(ENV_PATH).exists():
    Path(ENV_PATH).write_text(textwrap.dedent("""\
        # TESTNET ONLY — never put real mainnet keys here
        SEPOLIA_RPC=
        PRIVATE_KEY=
        ACCOUNT_ADDRESS=
    """))
    raise SystemExit(f"[init] Template .env created at {ENV_PATH}. Paste your MetaMask TEST private key + RPC, then re-run.")
load_dotenv(ENV_PATH, override=True)
SEPOLIA_RPC = (os.getenv("SEPOLIA_RPC") or "").strip()
PRIVATE_KEY  = (os.getenv("PRIVATE_KEY")  or "").strip()
ACCOUNT_ADDR = (os.getenv("ACCOUNT_ADDRESS") or "").strip()

In [18]:
def _fix_pk(pk: str) -> str:
    pk = (pk or "").strip().replace('"','').replace("'","").replace(" ","")
    if not pk.startswith("0x"): pk = "0x" + pk
    if len(pk) != 66 or not re.fullmatch(r"0x[0-9a-fA-F]{64}", pk):
        raise ValueError("PRIVATE_KEY must be 0x + 64 hex chars (export from MetaMask TEST account).")
    return pk
if not PRIVATE_KEY:
    raise ValueError("PRIVATE_KEY missing in .env. Export from MetaMask (TEST account), paste into .env, re-run.")
PRIVATE_KEY = _fix_pk(PRIVATE_KEY)
acct = Account.from_key(PRIVATE_KEY)
if not ACCOUNT_ADDR or ACCOUNT_ADDR.lower() != acct.address.lower():
    ACCOUNT_ADDR = acct.address
    set_key(ENV_PATH, "ACCOUNT_ADDRESS", ACCOUNT_ADDR)
candidates = [SEPOLIA_RPC] + RPC_CANDIDATES if SEPOLIA_RPC else RPC_CANDIDATES

In [19]:
def connect_any(urls):
    last_err = None
    for url in urls:
        if not url: 
            continue
        try:
            w3 = Web3(Web3.HTTPProvider(url, request_kwargs={"timeout": 20}))
            if w3.is_connected():
                return w3, url
        except Exception as e:
            last_err = e
    raise RuntimeError(f"No working Sepolia RPC. Set SEPOLIA_RPC in {ENV_PATH}. Last error: {last_err}")
w3, ACTIVE_RPC = connect_any(candidates)
if ACTIVE_RPC != SEPOLIA_RPC:
    set_key(ENV_PATH, "SEPOLIA_RPC", ACTIVE_RPC)
print(f"[rpc] Connected → {ACTIVE_RPC}")
print(f"[acct] Deployer: {ACCOUNT_ADDR}")

[rpc] Connected → https://ethereum-sepolia-rpc.publicnode.com
[acct] Deployer: 0xd437f22b71CB94f1A9724d3A60a98B08f825C1a8


In [20]:
def get_fee_params(w3: Web3):
    try:
        base = w3.eth.get_block("latest")["baseFeePerGas"]
        max_priority = w3.to_wei("1.5", "gwei")
        max_fee = int(base * 2 + max_priority)
    except Exception:
        max_priority = w3.to_wei("1.5", "gwei")
        max_fee = w3.to_wei("30", "gwei")
    return max_fee, max_priority
max_fee, max_priority = get_fee_params(w3)
eth = lambda n: w3.from_wei(n, "ether")
bal_wei = w3.eth.get_balance(ACCOUNT_ADDR)
cap_wei = GAS_LIMIT_DEPLOY * max_fee    
print(f"[preflight] Balance: {eth(bal_wei)} ETH | Worst-case deploy cap: ~{eth(cap_wei)} ETH")

[preflight] Balance: 0.05 ETH | Worst-case deploy cap: ~0.000752063927 ETH


In [21]:
def wait_for_funds(required_wei, check_every=POLL_EVERY_SECS):
    print(f"[wait] Need ≥ ~{eth(required_wei)} ETH on Sepolia.")
    while True:
        b = w3.eth.get_balance(ACCOUNT_ADDR)
        print(f"  → {ACCOUNT_ADDR} has {eth(b)} ETH")
        if b >= required_wei:
            print("[wait] Funds detected.")
            return b
        if not AUTO_WAIT_FOR_FUNDS:
            raise RuntimeError("Insufficient funds. Faucet some Sepolia ETH to this address, then re-run.")
        time.sleep(check_every)
if bal_wei < cap_wei:
    bal_wei = wait_for_funds(cap_wei)

In [22]:
vy_src = """
# @version 0.3.10
stored: public(uint256)
@external
def __init__(x: uint256):
    self.stored = x
@external
def set(x: uint256):
    self.stored = x
"""
compiled = compile_codes({"Store.vy": vy_src}, output_formats=["abi","bytecode"], interface_codes={})
abi = compiled["Store.vy"]["abi"]
bytecode = compiled["Store.vy"]["bytecode"]
print(f"[vyper] ABI entries: {len(abi)} | bytecode bytes: {len(bytecode)}")

[vyper] ABI entries: 3 | bytecode bytes: 304


In [23]:
def send_raw(w3: Web3, signed_obj):
    raw = getattr(signed_obj, "rawTransaction", None)
    if raw is None:
        raw = getattr(signed_obj, "raw_transaction", None)
    if raw is None and hasattr(signed_obj, "raw"):
        raw = signed_obj.raw
    if raw is None:
        raise RuntimeError("SignedTransaction has no raw bytes (check web3/eth-account versions).")
    return w3.eth.send_raw_transaction(raw)

In [24]:
Store = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.get_transaction_count(ACCOUNT_ADDR)
unsigned = Store.constructor(123).build_transaction({
    "chainId": CHAIN_ID,
    "from": ACCOUNT_ADDR,
    "nonce": nonce,
    "maxFeePerGas": max_fee,
    "maxPriorityFeePerGas": max_priority,
    "gas": GAS_LIMIT_DEPLOY,   
})

In [25]:
try:
    est = w3.eth.estimate_gas(unsigned)
    unsigned["gas"] = int(est * 1.2)
    print("[deploy] Refined gas:", unsigned["gas"])
except Exception:
    print("[deploy] Using GAS_LIMIT_DEPLOY:", unsigned["gas"])
signed = acct.sign_transaction(unsigned)
tx_hash = send_raw(w3, signed)
print("[deploy] tx:", tx_hash.hex())
rc = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=240)
if rc.status != 1:
    raise RuntimeError("Deployment failed (status != 1).")
STORE_ADDR = rc.contractAddress
print("[deploy] Contract @", STORE_ADDR)

[deploy] Refined gas: 117447
[deploy] tx: e6ef1ecdc8613fa28133e92632cd9153b039f021658be758ec83d78734c55c8d
[deploy] Contract @ 0x4eD937C4fcD3830E2dc6F5503dfF6E3b074D6623


In [26]:
store = w3.eth.contract(address=STORE_ADDR, abi=abi)
print("[call] stored() before:", store.functions.stored().call())
nonce2 = w3.eth.get_transaction_count(ACCOUNT_ADDR)
tx = store.functions.set(999).build_transaction({
    "chainId": CHAIN_ID,
    "from": ACCOUNT_ADDR,
    "nonce": nonce2,
    "maxFeePerGas": max_fee,
    "maxPriorityFeePerGas": max_priority,
    "gas": GAS_LIMIT_WRITE,  
})
signed2 = acct.sign_transaction(tx)
h = send_raw(w3, signed2)
print("[write] tx:", h.hex())
rc2 = w3.eth.wait_for_transaction_receipt(h, timeout=240)
if rc2.status != 1:
    raise RuntimeError("set() failed (status != 1).")
print("[call] stored() after:", store.functions.stored().call())
print("Sepolia deploy + interaction complete (signed by your MetaMask account).")

[call] stored() before: 123
[write] tx: 30baf508aa32a9f661b2a6e1ca62b188143db2ca5491a41cc7c69ca732195421
[call] stored() after: 999
Sepolia deploy + interaction complete (signed by your MetaMask account).


In [27]:
os.environ["ETH_RPC"] = ACTIVE_RPC
os.environ["MOCK_MODE"] = "0"
print("[runtime] ETH_RPC set for your app:", os.environ["ETH_RPC"])

[runtime] ETH_RPC set for your app: https://ethereum-sepolia-rpc.publicnode.com
